# **Project Name** - Agentic AI-Based Travel Planning Assistant Using LangChain

# **Project Type** - AI / Machine Learning Project – Agentic AI Application

### This is an end-to-end AI application that combines:

* #### Large Language Models (LLMs)

* #### Agentic AI (tool-based reasoning)

* #### API integration

* #### Interactive web application using Streamlit

### The project focuses on decision-making automation, not just prediction.

# **Contribution -** Individual Project 

### My contributions include:

* ####  Designing the overall system architecture

* #### Preparing and integrating structured datasets (flights, hotels, places)

* #### Building modular tools for data retrieval and analysis

* #### Implementing an agentic decision layer to plan trips intelligently

* #### Integrating real-time weather data using an external API

* #### Developing a professional Streamlit user interface

* #### Generating structured and human-readable final outputs

# **GitHub Link -** 

### https://github.com/tonujaramesh/Agentic-AI-Based-Travel-Planning-Assistant-Using-LangChain

# **Project Summary -** 

### The Agentic AI-Based Travel Planning Assistant is an intelligent system that automatically creates personalized travel itineraries based on user preferences such as source, destination, travel duration, and budget.

### The system uses agentic AI reasoning to:

* #### Select optimal flights and hotels

* #### Recommend attractions

* #### Analyze weather conditions

* #### Estimate total travel costs

* #### Generate a complete day-wise travel plan

#### By combining structured datasets, real-time APIs, and AI reasoning, the system delivers accurate, explainable, and user-friendly travel plans, similar to modern online travel platforms.

# **Problem Statement -** 

### Planning a trip manually is time-consuming and inefficient. Travelers often need to:

* #### Visit multiple websites for flights, hotels, and attractions

* #### Compare inconsistent information

* #### Manually estimate costs and schedules

* #### Handle weather uncertainty

#### This process can result in poor planning decisions, higher costs, and user frustration.There is a need for an intelligent, automated system that can reason like a travel expert and provide optimized, personalized itineraries in one place.

# **Business Use Cases -**

### The AI Travel Planning Assistant can be used by:

* #### **Online Travel Agencies (OTAs)** – to automate itinerary generation

* #### **Airline & Hotel Platforms** – to provide bundled travel plans

* #### **Tourism Boards** – to promote destinations intelligently

* #### **Corporate Travel Management** – for fast business trip planning

* #### **Self-service Travel Apps** – reducing dependency on customer support

### **Business Benefits :**

* #### Reduces customer support workload

* #### Saves user time and money

* #### Improves customer satisfaction

* #### Enables personalized recommendations

* #### Scales travel planning without human agents

#### Companies like MakeMyTrip, Booking.com, Cleartrip, and Ixigo are increasingly adopting similar agentic and conversational AI systems.

# **DATA SETUP CODE**

## *Importing Libaries*

In [5]:
import json
from pathlib import Path
import os

# Works in both Jupyter Notebook and .py files
try:
    BASE_DIR = Path(__file__).resolve().parent.parent
except NameError:
    BASE_DIR = Path(os.getcwd())

DATA_DIR = BASE_DIR / "Data_Sources"

def load_json(filename):
    path = DATA_DIR / filename
    if not path.exists():
        raise FileNotFoundError(f"{path} not found")
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# Load datasets
flights_data = load_json("flights.json")
hotels_data = load_json("hotels.json")
places_data = load_json("places.json")


# **BUILD LANGCHAIN TOOLS**

### “LangChain tools allow the agent to interact with structured datasets and external APIs. In this project, tools were used for flight selection, hotel recommendation, attraction discovery, weather analysis, and budget estimation, enabling the agent to make intelligent, explainable travel-planning decisions.”

### **1. Flight Search Tool**

#### i. Filters by source → destination

#### ii. Suggests cheapest / fastest flight

In [6]:
from langchain.tools import tool
from datetime import datetime
from tools.data_loader import flights_data

@tool
def flight_search_tool(source: str, destination: str):
    """Find cheapest and fastest flight between two cities"""

    matches = [
        f for f in flights_data
        if f["from"].lower() == source.lower()
        and f["to"].lower() == destination.lower()
    ]

    if not matches:
        return {"message": "No flights found"}

    for f in matches:
        dep = datetime.fromisoformat(f["departure_time"])
        arr = datetime.fromisoformat(f["arrival_time"])
        f["duration_hours"] = round((arr - dep).seconds / 3600, 2)

    cheapest = min(matches, key=lambda x: x["price"])
    fastest = min(matches, key=lambda x: x["duration_hours"])

    return {
        "cheapest_flight": cheapest,
        "fastest_flight": fastest
    }


### **2. Hotel Recommendation Tool**

#### i. Reads hotels.json

#### ii. Filters by city, rating, price

In [7]:
from langchain.tools import tool
from tools.data_loader import hotels_data

@tool
def hotel_recommendation_tool(city: str, min_stars: int = 3, max_price: int = None):
    """Recommend hotels based on city, rating, and price"""

    results = [
        h for h in hotels_data
        if h["city"].lower() == city.lower()
        and h["stars"] >= min_stars
    ]

    if max_price:
        results = [h for h in results if h["price_per_night"] <= max_price]

    results.sort(key=lambda x: (-x["stars"], x["price_per_night"]))

    return results[:3] if results else {"message": "No hotels found"}


### **3. Places Discovery Tool**

#### i. Reads places.json

#### ii. Recommends attractions based on type & rating

In [8]:
from langchain.tools import tool
from tools.data_loader import places_data

@tool
def places_discovery_tool(city: str, place_type: str = None, min_rating: float = 4.0):
    """Recommend attractions based on type and rating"""

    results = [
        p for p in places_data
        if p["city"].lower() == city.lower()
        and p["rating"] >= min_rating
    ]

    if place_type:
        results = [p for p in results if p["type"].lower() == place_type.lower()]

    results.sort(key=lambda x: -x["rating"])
    return results[:5] if results else {"message": "No attractions found"}


### **4. Weather Lookup Tool (REAL API)**

#### i. Calls free Open-Meteo API

#### ii. Provides forecast for travel dates

In [9]:
from langchain.tools import tool
import requests

@tool
def weather_lookup_tool(latitude: float, longitude: float):
    """Get real-time weather from Open-Meteo API"""

    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        "&current_weather=true"
        "&daily=temperature_2m_max,temperature_2m_min"
        "&timezone=auto"
    )

    data = requests.get(url, timeout=10).json()

    return {
        "current_temperature": data["current_weather"]["temperature"],
        "wind_speed": data["current_weather"]["windspeed"],
        "max_temp": data["daily"]["temperature_2m_max"][0],
        "min_temp": data["daily"]["temperature_2m_min"][0]
    }


### **5. Budget Estimation Tool**

#### i. Sums flight + hotel + daily expenses

In [10]:
from langchain.tools import tool

@tool
def budget_estimation_tool(
    flight_price: int,
    hotel_price_per_night: int,
    nights: int,
    daily_local_expense: int = 1500
):
    """Estimate total trip budget"""

    hotel_cost = hotel_price_per_night * nights
    local_cost = daily_local_expense * nights
    total = flight_price + hotel_cost + local_cost

    return {
        "flight_cost": flight_price,
        "hotel_cost": hotel_cost,
        "local_expenses": local_cost,
        "total_budget": total
    }


# ***CREATE THE AGENT (LANGCHAIN)***

### The goal is to enable the system to autonomously solve complex tasks that require:

* #### Multiple data sources

* #### Sequential reasoning

* #### Decision-making under constraints

* #### Explainable outcomes

### In the travel planning context, the agent behaves like a virtual travel expert.

In [11]:
import importlib
import tools.flight_tool as ft

importlib.reload(ft)

print("File:", ft.__file__)
print("Has function:", hasattr(ft, "flight_search_tool"))
print("Dir contains function:", "flight_search_tool" in dir(ft))


File: c:\Users\srame\OneDrive\Desktop\Internship_Labmentix\Project-8_Travel_Planner\tools\flight_tool.py
Has function: True
Dir contains function: True


In [3]:
from tools.flight_tool import flight_search_tool
print("✅ flight_search_tool imported successfully")
from tools.flight_tool import flight_search_tool
flight_search_tool.invoke({
    "source": "Bangalore",
    "destination": "Goa"
})


✅ flight_search_tool imported successfully


{'cheapest_flight': {'flight_id': 'FL0021',
  'airline': 'Go First',
  'from': 'Bangalore',
  'to': 'Goa',
  'departure_time': '2025-02-06T07:27:00',
  'arrival_time': '2025-02-06T09:27:00',
  'price': 5356,
  'duration_hours': 2.0},
 'fastest_flight': {'flight_id': 'FL0020',
  'airline': 'Go First',
  'from': 'Bangalore',
  'to': 'Goa',
  'departure_time': '2025-05-20T07:24:00',
  'arrival_time': '2025-05-20T08:24:00',
  'price': 6624,
  'duration_hours': 1.0}}

## **INSIGHTS**

### **1️⃣ Tool-Based Execution Enables Reliability**

#### Using .invoke() ensures the function runs with validated inputs and structured outputs, reducing runtime errors and making the system more reliable than direct function calls.

### **2️⃣ Separation of Reasoning and Action**

### The tool cleanly separates:

* #### Reasoning (handled by the agent/LLM)

* #### Action (handled by the tool logic)

#### This design aligns with agentic AI principles in LangChain.

### **3️⃣ Deterministic Outputs Improve Trust**

#### Unlike LLM text generation, the tool:

* #### Reads real data (flights.json)

* #### Applies deterministic filters (source → destination)

* #### Computes durations and prices programmatically

#### This improves accuracy, repeatability, and user trust.

### **4️⃣ Pre-Agent Testing Improves System Stability**

### Invoking the tool independently proves:

* #### The dataset is correctly loaded

* #### Business logic works as intended

* #### The tool is agent-ready

#### This practice reduces downstream agent failures.

### **5️⃣ Supports Multi-Objective Optimization**

### The tool returns both:

* #### Cheapest flight (cost optimization)

* #### Fastest flight (time optimization)

#### This enables the agent to balance constraints based on user preferences—mirroring real travel platforms.

In [ ]:
# 1️⃣ Hotel Tool
import importlib
import tools.hotel_tool as ht
importlib.reload(ht)

print("Hotel:", hasattr(ht, "hotel_recommendation_tool"))

# 2️⃣ Places Tool
import tools.place_tool as pt
importlib.reload(pt)

print("Places:", hasattr(pt, "places_discovery_tool"))

# 3️⃣ Weather Tool
import tools.weather_tool as wt
importlib.reload(wt)

print("Weather:", hasattr(wt, "weather_lookup_tool"))

# 4️⃣ Budget Tool
import tools.budget_tool as bt
importlib.reload(bt)

print("Budget:", hasattr(bt, "budget_estimation_tool"))

Hotel: True
Places: True
Weather: True
Budget: True


# ***Final Output***

In [5]:
from agent.travel_agent import run_travel_agent
from output.final_output import generate_final_output, print_expected_results

agent_result = run_travel_agent({
    "source": "Bangalore",
    "destination": "Goa",
    "days": 4,
    "budget": 3000,
    "latitude": 15.2993,
    "longitude": 74.1240
})

final_output = generate_final_output(agent_result, days=4)
print_expected_results(final_output)


Your 4 Days Trip to Goa

✈️ Flight Selected:
- Go First (₹5356) – Departs Bangalore at 07:27

🏨 Hotel Booked:
- Royal Heritage (₹1232/night, 5-star)

🌦️ Weather:
- Day 1: Max 33.5°C, Min 19.0°C
- Day 2: Max 33.5°C, Min 19.0°C
- Day 3: Max 33.5°C, Min 19.0°C
- Day 4: Max 33.5°C, Min 19.0°C

🗺️ Itinerary:
Day 1: Famous Park
Day 2: Beautiful Park
Day 3: Popular Lake
Day 4: Historic Park

💰 Estimated Total Budget:
- Flight: ₹5356
- Hotel: ₹4928
- Food & Travel: ₹6000
-------------------------------------
Total Cost: ₹16284


# **Streamlit App**

### **🧭 User-Centric Interface**

#### Designed with intuitive inputs and clear sections, making trip planning easy even for non-technical users.

### **🎛️ Interactive Controls**

#### Sliders, text fields, and buttons allow users to customize travel preferences in real time.

### **🧩 Clean & Minimal Layout**

#### A minimalist design ensures better readability, faster navigation, and a smooth user experience.

### **⚡ Real-Time Responsiveness**

#### Instant feedback with loaders and live results improves user trust and engagement.

### **📊 Structured Output Display**

#### Travel plans are presented in an organized format (summary, itinerary, weather, budget) for easy understanding.

# **Conclusion**

### The **Agentic AI-Based Travel Planning Assistant Using LangChain** successfully demonstrates how **agentic artificial intelligence** can be applied to solve real-world problems in the travel and tourism domain. The project integrates structured datasets, real-time weather APIs, and intelligent tool-based reasoning to automatically generate personalized travel itineraries.

### Through a modular design, the system efficiently performs flight selection, hotel recommendation, attraction discovery, weather analysis, and budget estimation. The use of **LangChain tools** enables deterministic data access, while the agentic decision layer ensures logical reasoning and explainable outputs. This approach moves beyond traditional chatbots and showcases **autonomous decision-making capabilities**.

### The Streamlit application provides a clean, interactive, and user-friendly interface, allowing users to customize trip preferences and instantly receive AI-generated travel plans. By combining backend intelligence with a minimalist frontend, the project delivers both **technical robustness** and **practical usability**.

### Overall, this project highlights the potential of agentic AI systems to reduce manual effort, improve decision quality, and enhance user experience. It serves as a strong foundation for future extensions such as real-time booking integrations, multi-city travel planning, and advanced personalization, making it a scalable and industry-relevant AI solution.